In [7]:
class MultiLayer:
  def __init__(self):
    self.x = None
    self.y = None
    
  def forward(self, x, y):
    self.x = x
    self.y = y
    out = x * y
    
    return out
  
  def backward(self, dout):
    dx = dout * self.y
    dy = dout * self.x
    
    return dx, dy

In [8]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MultiLayer()
mul_tax_layer = MultiLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [9]:
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [10]:
class AddLayer:
  def __init__(self):
    pass
  
  def forward(self, x, y):
    out = x + y
    return out
  
  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1
    
    return dx, dy

In [12]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1


mul_apple_layer = MultiLayer()
mul_orange_layer = MultiLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MultiLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
dorange, dorange_num = mul_apple_layer.backward(dorange_price)

print(price)
print(dapple, dapple_num, dorange, dorange_num, dtax)


715.0000000000001
2.2 110.00000000000001 2.2 110.00000000000001 650


In [13]:
class Relu:
  def __init__(self):
    self.mask = None
    
  def forward(self, x):
    self.mask = (x <= 0)
    out = x.copy()
    out[self.mask] = 0
    
    return out
    
  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout
    
    return dx

In [15]:
import numpy as np

x = np.array([[1.0, -0.5], 
              [-2.0, 3.0]])

print(x)
relu = Relu()
relu.forward(x)

[[ 1.  -0.5]
 [-2.   3. ]]


array([[1., 0.],
       [0., 3.]])

In [16]:
class Sigmoid:
  def __init__(self):
    self.out = None
    
  def forward(self, x):
    out = np.piecewise(
        x,
        [x > 0],
        [lambda i: 1 / (1 + np.exp(-i)), lambda i: np.exp(i) / (1 + np.exp(i))],
    )
    self.out = out
    
    return out
  
  def backward(self, dout):
    dx = dout * (1.0 - self.out) * self.out
    
    return dx

In [18]:
class Affine:
  def __init__(self, W, b):
    self.W = W
    self.b = b
    self.x = None
    self.dW = None
    self.db = None
    
  def forward(self, x):
    self.x = x
    out = np.dot(x, self.W) + self.b
    
    return out
  
  def backward(self, dout):
    dx = np.dot(dout, self.W.T)
    self.dW = np.dot(self.x.T, dout)
    self.db = np.sum(dout, axis=0)
    
    return dx

In [16]:
class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None
    self.y = None
    self.t = None
  
  def softmax(self, a):
    C = np.max(a)
    exp_a = np.exp(a - C)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y
  
  def cross_entropy_error(self, y, t, is_onehot=True):
    if y.ndim == 1:
      t = t.reshape(1, t.size)
      y = y.reshape(1, y.size)
      
    batch_size = y.shape[0]
    if is_onehot:
      return -np.sum(t * np.log(y + 1e-7)) / batch_size 
    else:
      return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size
    
  def forward(self, x, t):
    self.t = t
    self.y = self.softmax(x)
    self.loss = self.cross_entropy_error(self.y, self.t)
    return self.loss
  
  def backward(self, dout=1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size
    
    return dx

만든 layer들로 TwoLayerNet 구현하기

In [1]:
import numpy as np
import sys
sys.path.append('../modules/')
from common.layers import *
from common.gradient import *
from collections import OrderedDict

class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
    # 가중치 초기화
    self.params = {}
    
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)
    
    # 계층 생성
    self.layers = OrderedDict()
    self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
    self.layers['Relu1'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
    
    self.lastLayer = SoftmaxWithLoss()
    
    
  def predict(self, x):
    for layer in self.layers.values():
      x = layer.forward(x)
      
    return x
  
  def loss(self, x, t):
    y = self.predict(x)
    return self.lastLayer.forward(y, t)
  
  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    
    if t.ndim != 1: t = np.argmax(t, axis=1)
    
    accuracy = np.sum(y == t) / float(x.shape[0])
    return accuracy 
  
  # for gradient check
  def numerical_gradient_check(self, x, t):
    loss_W = lambda W: self.loss(x, t)
    
    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

    return grads

  def gradient(self, x, t):
    self.loss(x, t)
    
    dout = 1
    dout = self.lastLayer.backward(dout)
    
    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
      dout = layer.backward(dout)
      
    grads = {}
    grads['W1'] = self.layers['Affine1'].dW
    grads['b1'] = self.layers['Affine1'].db
    grads['W2'] = self.layers['Affine2'].dW
    grads['b2'] = self.layers['Affine2'].db
    
    return grads

In [2]:
# gradient check
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient_check(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
  diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
  print(f"{key} : {diff}")

W1 : 9.857621622188434e-05
b1 : 0.0007801609363680561
W2 : 0.0022673608552423144
b2 : 0.0666666667867928


In [4]:
import warnings
warnings.filterwarnings('ignore')

# 학습 구현하기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]
  
  grad = network.gradient(x_batch, t_batch)
  
  for key in ('W1', 'b1', 'W2', 'b2'):
    network.params[key] -= learning_rate * grad[key]
    
  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)
  
  if i % iter_per_epoch == 0:
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    print(f"train set : {train_acc:.4f} / test set: {test_acc:.4f}")


train set : 0.1178 / test set: 0.1154
train set : 0.0975 / test set: 0.0974
train set : 0.0986 / test set: 0.0958
train set : 0.0975 / test set: 0.0974
train set : 0.0987 / test set: 0.0980
train set : 0.0987 / test set: 0.0980
train set : 0.0987 / test set: 0.0980
train set : 0.0987 / test set: 0.0980
train set : 0.0987 / test set: 0.0980
train set : 0.0987 / test set: 0.0980
train set : 0.0987 / test set: 0.0980
train set : 0.0987 / test set: 0.0980
train set : 0.0987 / test set: 0.0980
train set : 0.0987 / test set: 0.0980
train set : 0.0987 / test set: 0.0980
train set : 0.0987 / test set: 0.0980
train set : 0.0987 / test set: 0.0980
